# Given Image Address in Share Directory, Generate Geometry output.
Split into binary geometry matrix and RI matrix, geometry matrix for meshing.

In [ ]:
# Import libraries:
import numpy as np
from PIL import Image 
import tiffile as tf
import scipy as sp
import os

## Set-up I/O Path Allocations 
Declare input data path and access tiff or tif. \
The path variable can be changed to output different datas' geometry arrays.

In [ ]:
# Storage Variables:
geo_subfolder_address = ".data/processed_geometry/"
RI_subfolder_address = ".data/processed_RI/"
raw_subfolder_address = ".data/raw/"

# Helpers:
def output_address(file_name, appendent="Raw", subfolder_address=".data/"):
    base_name = file_name.split(".")[0]
    out_path = f"{subfolder_address}{base_name}_{appendent}.mat"
    return out_path

# Declarables: 
file_name = "________"     # Replace with actual file name
access_path = f"{raw_subfolder_address}{file_name}"

## Define Important Parameters
Declare Threshold Value for solid existance
Declare Sizings for the solid model

In [ ]:
# Apply RI Threshold to mask the image:
ri_threshold = 1.33     # Can be changed

# Load raw file:
if(file_name.split(".")[-1] == "tiff"):
    raw = tf.imread(access_path)
    print(f"TIFF loaded successfully!")
elif(file_name.split(".")[-1] == "jpeg" or file_name.split(".")[-1] == "jpg" or file_name.split(".")[-1] == "png"):
    raw = Image.open(access_path)
    print(f"Image loaded successfully!")
else:
    raise ValueError("Unsupported file format. Please use TIFF, JPEG, or PNG.")

In [ ]:
# Convert to numpy array (already a numpy matrix)
try:
    raw_mat = np.array(raw)
except Exception as e:
    print(f"Error converting image to numpy array: {e}")
    raise

# Print raw file information:
print(f"Shape: {raw_mat.shape}")
print(f"Data type: {raw_mat.dtype}")
print(f"Min value: {raw_mat.min()}, Max value: {raw_mat.max()}")
print(raw_mat)  # Print the raw matrix to verify its contents

# Extraction and Processing
Move to process the raw tiff file, return binary mask of matrix and save the work in allocated output location.

In [ ]:
binary_mat = np.where(raw_mat >= ri_threshold, 1, 0)
print(f"Binary matrix created with threshold {ri_threshold}.\n")
print(f"Shape: {binary_mat.shape}\n")
print(f"Unique values in binary matrix: {np.unique(binary_mat)}\n")
print(binary_mat) # Print the binary matrix to verify its contents